# **Text summarization with Attention**
- abstractive summarization
- used Amazon review data

## **Understanding Amazon review data**

In [ ]:
! pip install ipykernel

In [ ]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing.text import Tokenizerfrom 
from tensorflow.keras.preprocessing.sequence import pad_sequences
import utllib.request
np.random.seed(seed=0)